# This notebook illustrates how to deploy a tuned model in watsonx.ai July 7, 2023 GA release
### #
#### Author: Shuvanker Ghosh - sghosh@us.ibm.com
####
#### In this example notebook, I have a tuned model endpoint, which I then deployed on watsonx.ai using python function wrapper approach for consumption of the model for generating use case specific content

### 1. Define the deployable function
Define a Python closure with an inner function named "score" which then connects to tuned model endpoint to get the generated text for a given input

In [ ]:
#wml_python_function
def my_deployable_function():
    import requests
    class FoundationModelWrapper:
        def __init__(self, model_end_point, apikey, model_id, model_parameters):
            self.model_end_point = model_end_point
            self.apikey = apikey
            self.model_id = model_id
            self.model_parameters = model_parameters

        def generate(self, model_input):
            
            # Access token may be needed for some cases
            # from ibm_cloud_sdk_core import IAMTokenManager
            # from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
            # access_token = IAMTokenManager(
            # apikey =  self.apikey,
            # url = "https://iam.cloud.ibm.com/identity/token"
            #).get_token()

            headers = {
                "Authorization": "Bearer " + self.apikey,
                "Content-Type": "application/json",
                "Accept": "application/json"
            }
            data = {
                "model_id": self.model_id,
                "inputs": [model_input],
                "parameters": self.model_parameters
            }
            response = requests.post(self.model_end_point, json=data, headers=headers)
            if response.status_code == 200:
                return response.json()["results"][0]["generated_text"]
            else:
                return response.text

    def score( payload ):
        import os

        apikey = "PASTE YOUR API KEY"

        # Set tuned model end point
        model_end_point = "PASTE YOU TUNED MODEL ENDPOINT"
        model_end_point = "https://workbench-api.res.ibm.com/v1/generate"
    
        
        # Set tuned model id
        model_id = "PASTE YOUR TUNED FOUNDATION MODEL ID"
        model_id = "flan-t5-xl-mpt-tcFvBXTp-2023-07-11-17-05-03"
        #model_id = "google/flan-ul2"
        
        # Set model parameters
        model_parameters = {
            "decoding_method": "greedy",
            "min_new_tokens": 1,
            "max_new_tokens": 200,
            "beam_width": 1
        }
        
        # Create the wrapper
        wrapper = FoundationModelWrapper(model_end_point, apikey, model_id, model_parameters)
        
        # Get the mode input from the payload
        model_input = payload.get("input_data")[0].get("values")[0][0]

        # Generate and capture model output
        model_output = wrapper.generate(model_input)
        
        # Score using the pre-defined model
        score_response = {
            'predictions': [{'fields': ['Response_message_field'], 
                             'values': [[model_output]]
                            }]
        } 
        return score_response
       

    return score

### 2. Test locally
Test your function in the notebook before deploying the function.

In [ ]:
# Pass the sample input to the function as a test

func_result = my_deployable_function()({"input_data": [{"values": [["My water cooler is not dispensing"]]}]})
print(func_result)

### 3. Set up deployment enviornment
Set up the WML environment to deploy the foundation model wrapper function
Pre requisite: Create a deployment space and note down the space_id

In [ ]:
# Setup API Key and WML credentials
apikey = 'PASTE YOUR IBM CLOUD API KEY'
location = 'us-south'

wml_credentials = {
    "apikey": apikey,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

from ibm_watson_machine_learning import APIClient

# WML client
client = APIClient(wml_credentials)

# Set space_id
space_id = 'PASTE YOUR SPACE ID HERE'

client.spaces.list(limit=20)

# Set default space
client.set.default_space(space_id)

### 4. Store and deploy the function
Before you can deploy the function, you must store the function in your Watson Machine Learning repository.

In [ ]:
# Look up software specification for the deployable function
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.2-py3.10")

model_name = "Service Request Categorization"


# Store the deployable function in your Watson Machine Learning repository
meta_data = {
    client.repository.FunctionMetaNames.NAME: f"{model_name} Function",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

function_details = client.repository.store_function(meta_props=meta_data, function=my_deployable_function)

# Get published function ID
function_uid = client.repository.get_function_uid(function_details)
print (function_uid)

# Deploy the stored function
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: f"{model_name} Function Deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)
print(function_deployment_details)

### 5. Test the deployed function
Use the Watson Machine Learning Python client to send data to your function deployment for processing in exactly the same way you send data to model deployments for processing.

In [ ]:
# Get the endpoint URL of the function deployment just created
function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
print(function_deployment_id)
print(function_deployment_endpoint_url)

# Test payload
payload = {"input_data": [{"values": [["Dryer is not heating up"]]}]}

# Score
result = client.deployments.score(function_deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)